# Introduction to DataFrames

https://devblogs.microsoft.com/dotnet/an-introduction-to-dataframe/

In [2]:
#r "nuget:Microsoft.Data.Analysis,0.2.0"
    using Microsoft.Data.Analysis;

Installed package Microsoft.Data.Analysis version 0.2.0

In [3]:
PrimitiveDataFrameColumn<DateTime> dateTimes = new PrimitiveDataFrameColumn<DateTime>("DateTimes"); // Default length is 0.
PrimitiveDataFrameColumn<int> ints = new PrimitiveDataFrameColumn<int>("Ints", 3); // Makes a column of length 3. Filled with nulls initially
StringDataFrameColumn strings = new StringDataFrameColumn("Strings", 3); // Makes a column of length 3. Filled with nulls initially

In [4]:
// Append 3 values to dateTimes
dateTimes.Append(DateTime.Parse("2019/01/01"));
dateTimes.Append(DateTime.Parse("2019/01/01"));
dateTimes.Append(DateTime.Parse("2019/01/02"));

In [5]:
DataFrame df = new DataFrame(dateTimes, ints, strings); // This will throw if the columns are of different lengths


In [6]:
df

Columns,Rows
"[ [ 2019-01-01 00:00:00Z, 2019-01-01 00:00:00Z, 2019-01-02 00:00:00Z ], [ <null>, <null>, <null> ], [ <null>, <null>, <null> ] ]","[ [ 2019-01-01 00:00:00Z, <null>, <null> ], [ 2019-01-01 00:00:00Z, <null>, <null> ], [ 2019-01-02 00:00:00Z, <null>, <null> ] ]"


In [7]:
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }

    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));

    writer.Write(t);
}, "text/html");


(2,1): warning CS0618: 'Formatter<DataFrame>.Register(Action<DataFrame, TextWriter>, string, bool)' is obsolete: 'Please use Formatter.Register<T>(formatter, ...)'



In [8]:
df

0,2019-01-01 00:00:00Z,<null>,<null>
1,2019-01-01 00:00:00Z,<null>,<null>
2,2019-01-02 00:00:00Z,<null>,<null>


## Charting

In [9]:
#r "nuget:MathNet.Numerics,4.9.0"
using XPlot.Plotly;
using System.Linq;
using MathNet.Numerics.Distributions;

double mean = 0;
double stdDev = 0.1;
MathNet.Numerics.Distributions.Normal normalDist = new Normal(mean, stdDev);

PrimitiveDataFrameColumn<double> doubles = new PrimitiveDataFrameColumn<double>("Normal Distribution", normalDist.Samples().Take(1000));
display(Chart.Plot(
    new Graph.Histogram()
    {
        x = doubles,
        nbinsx = 30
    }
));